This code implements the referencing with a neural network when using a reference spectrum in transient absorption spectroscopy. The script performs the following operations

      (1) Setting up a neural network model

      (2) Training (fitting) the model with data of the reference and signal spectrum without the pump. The dataset is split in training and test set to monitor the performance.
   
      (3) Using the fitted model to perform de-noising on pump probe data


Inputs (required):

      signal_pump_raw & signal_Npump_raw = signal spectra for pump on and off 
            size (Npixel x Ndelays x Nacquisition)
   
      reference_pump_raw & reference_Npump_raw = reference spectra for pump on and off 
            size (Npixel x Ndelays x Nacquisition)

      x_data = Set of refecense spectra without the pump, e.g. reference_Npump_raw at every time delay
            size (Nacquisition x Npixel), as numpy array

      y_data = Set of signal spectra without the pump, e.g. signal_Npump_raw at every time delay
            size (Nacquisition x Npixel), as numpy array

      For optimal performance of the neural network, all values need to be scaled in the range of 0 to 1! 

            All signal spectra devided by thier maximum and all reference spectra devided by thier maximum.
Outputs:

      dOD_list = Delta OD for every delay and pixel
            size(Ndelays x Npixel)

      Error_list = error of the mean
            size(Ndelays x Npixel)



Citation for this code or some of its parts: T. Gutberlet et al., High-sensitivity extreme-ultraviolet transient absorption spectroscopy enabled by machine learning, (https://doi.org/10.1364/OE.495821)

In [ ]:
#import, using Keras with Tensorflow backend
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import numpy as np
import scipy.io
import h5py

#provide inputs, for example:
hf = h5py.File('data.h5', 'r')
signal_pump_raw=np.array(hf.get('P_Sig_raw'))
signal_Npump_raw=np.array(hf.get('N_Sig_raw'))
reference_pump_raw=np.array(hf.get('P_Ref_raw'))
reference_Npump_raw=np.array(hf.get('N_Ref_raw'))
x_data=np.array(hf.get('x_data'))
y_data=np.array(hf.get('y_data'))
hf.close()

# (1) Define neural network:

N_in  = np.shape(x_data)[1]  # Number of in neurons (reference)
N_out = np.shape(y_data)[1]  # Number of out neurons (signal)
model_simple = Sequential()  
# 3 hidden layers with relu activation function, output layer with linear activation
#200,100,20 neurons suited for ~10-20 decreete harmonics
#250, 100, 70 neurons suited for 1024 pixel
model_simple.add(Dense(200, activation='relu', input_shape=(N_in,)))
model_simple.add(Dense(100, activation='relu'))
model_simple.add(Dense(20, activation='relu'))
model_simple.add(Dense(N_out, activation='linear'))  
model_simple.summary()
#gradient decent with adam, mean squares loss function
model_simple.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model_simple.save_weights('model_init.h5') # initiate 'empty' weigts (fitting parameters) 

In [ ]:
# (2) train the neural network 

model_simple.load_weights('model_init.h5') # load 'empty' model
random_numer_param=685 # Random value
split_ratio=0.2 # Training test split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=split_ratio, random_state=random_numer_param)
batch_size = 50 
# EarlyStopping: the model should stop training when it won't improve anymore
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=50)
epochs = 10000 # max itterations
# train the model
history_simple = model_simple.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[early_stopping_monitor],validation_data=(x_test, y_test))  
#save model weights
FileName='Trained-Weights'
model_simple.save_weights(FileName +'.h5' )
# evaluate model performance
score = model_simple.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])

In [3]:
# (3) evaluate pump probe data: calculate the transient transmission

# load model weights
FileName='Trained-Weights'
model_simple.load_weights(FileName +'.h5' )

#Outputs
dOD_list=np.zeros((np.shape(signal_pump_raw)[1],np.shape(signal_pump_raw)[0])) # DeltaOD, size(Ndelays x Npixel)
Error_list=np.zeros((np.shape(signal_pump_raw)[1],np.shape(signal_pump_raw)[0])) #Std, size(Ndelays x Npixel)

for jj in range(np.shape(signal_pump_raw)[1]): # iterate delays
    signal_pump=np.transpose(signal_pump_raw[:,jj,:])
    signal_Npump=np.transpose(signal_Npump_raw[:,jj,:])
    reference_pump=np.transpose(reference_pump_raw[:,jj,:])
    reference_Npump=np.transpose(reference_Npump_raw[:,jj,:])
    # evaluate the model
    signal_Npump_predicted = model_simple.predict(reference_Npump)
    signal_pump_predicted = model_simple.predict(reference_pump)
    signal_pump_referenced=signal_pump/signal_pump_predicted # reference pumped spectrum
    signal_Npump_referenced=signal_Npump/signal_Npump_predicted # reference un pumped spectrum
    #compute delta OD & standart deviation
    for ii in range(np.shape(signal_pump_raw)[0]): # iterate pixel           
        dOD=-np.log10(2*(signal_pump_referenced[:,ii]-signal_Npump_referenced[:,ii])/(signal_pump_referenced[:,ii]+signal_Npump_referenced[:,ii])+1) # delta OD
        dOD_list[jj,ii]=np.mean(dOD)     
        Error_list[jj,ii]=np.std(dOD)/np.sqrt(np.shape(dOD)[0]) # error of the mean   